In [3]:
from os import path
import graphlab as gl
from datetime import datetime

## my key for graphlab
gl.product_key.set_product_key('49FA-0266-3006-D865-C0D5-2C05-56B6-4129')


# Load Data

MovieLens dataset collected by the GroupLens Research Project at the University of Minnesota.
For more information, see http://grouplens.org/datasets/movielens/

In [4]:
# Path to the dataset directory
data_dir = './dataset/ml-20m'

# Table of movies we are recommending: movieId, title, genres
items = gl.SFrame.read_csv(path.join(data_dir, 'movies.csv'))

# Table of interactions between users and items: userId, movieId, rating, timestamp
actions = gl.SFrame.read_csv(path.join(data_dir, 'ratings.csv'))

This non-commercial license of GraphLab Create for academic use is assigned to rahulbali.mecse16@pec.edu.in and will expire on November 23, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\cramdog\AppData\Local\Temp\graphlab_server_1528021934.log.0


Finished parsing file E:\ACADEMICS\PEC---\ME 4 SEM\code\graph-code\dataset\ml-20m\movies.csv

Parsing completed. Parsed 100 lines in 0.104096 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,float,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2042753 lines. Lines per second: 875547

Read 11940935 lines. Lines per second: 1.62602e+006

Finished parsing file E:\ACADEMICS\PEC---\ME 4 SEM\code\graph-code\dataset\ml-20m\ratings.csv

Parsing completed. Parsed 20000263 lines in 10.8368 secs.

# Prepare Data

In [5]:
# Prepare the data by removing items that are rare
rare_items = actions.groupby('movieId', gl.aggregate.COUNT).sort('Count')
rare_items = rare_items[rare_items['Count'] <= 5]
items = items.filter_by(rare_items['movieId'], 'movieId', exclude=True)
actions = actions.filter_by(rare_items['movieId'], 'movieId', exclude=True)
actions['timestamp'] = actions['timestamp'].astype(datetime)

In [6]:
# Extract year, title, and genre
items['year'] = items['title'].apply(lambda x: x[-5:-1])
items['title'] = items['title'].apply(lambda x: x[:-7])
items['genres'] = items['genres'].apply(lambda x: x.split('|'))

# Train Recommender Model

In [7]:
training_data, validation_data = gl.recommender.util.random_split_by_user(actions, 'userId', 'movieId')
model = gl.recommender.create(training_data, 'userId', 'movieId')

Recsys training: model = item_similarity

Warning: Ignoring columns rating, timestamp;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 1m 34s                              | 100              | 17519           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 96.5337s

# Explore and Evaluate the Model

In [8]:
# Get the metadata ready
urls = gl.SFrame.read_csv(path.join(data_dir, 'movie_urls.csv'))
items = items.join(urls, on='movieId')
users = gl.SFrame.read_csv(path.join(data_dir, 'user_names.csv'))

Could not detect types. Using str for each column.


RuntimeError: Runtime Exception. No files corresponding to the specified path (E:\ACADEMICS\PEC---\ME 4 SEM\code\graph-code\dataset\ml-20m\movie_urls.csv).

In [ ]:
# Interactively evaluate and explore recommendations
view = model.views.overview(validation_set=validation_data,
                            user_data=users,
                            item_data=items,
                            item_name_column='title',
                            item_url_column='url')
view.show()